In [1]:
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt
import math
from functools import partial
from open3d.t.geometry import TriangleMesh
from tqdm import tqdm 

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
dataname = "C:/Users/chris/Desktop/NewData/CoverCleaned290k.ply"
pcd = o3d.io.read_point_cloud(dataname)

In [3]:
pcd_center = pcd.get_center()
pcd.translate(-pcd_center)

PointCloud with 288918 points.

In [4]:
#Outlier removal

nn = 16
std_multiplier = 10

filtered_pcd = pcd.remove_statistical_outlier(nn,std_multiplier)
outliers = pcd.select_by_index(filtered_pcd[1], invert = True)
outliers.paint_uniform_color([1,0,0])
filtered_pcd = filtered_pcd[0]

#o3d.visualization.draw_geometries([filtered_pcd, outliers])

In [5]:
#Downsampling

voxel_size = 0.01
pcd_downsampled = filtered_pcd.voxel_down_sample(voxel_size=voxel_size)
print(f'number of points: {len(pcd_downsampled.points)}')
#o3d.visualization.draw_geometries([pcd_downsampled])

number of points: 288904


In [6]:
#Extract normals

nn_distance = np.mean([pcd.compute_nearest_neighbor_distance()])
radius_normals = nn_distance*4

pcd_downsampled.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normals, max_nn=16), fast_normal_computation=True)

pcd_downsampled.paint_uniform_color([0.6,0.6,0.6])
#o3d.visualization.draw_geometries([pcd_downsampled, outliers])

PointCloud with 288904 points.

In [7]:
# Extracting and Setting Parameters

front =  [-0.47452876114542436, 0.57451207113849134, -0.66690204300328082]
lookat = [-6.3976792217838847, 20.927374714553928, 18.659758576873813]
up =  [-0.056918726368614558, -0.77607794684805009, -0.62806311705487861]
zoom = 0.69999999999999996

pcd = pcd_downsampled

#o3d.visualization.draw_geometries([pcd_downsampled], zoom=zoom, front=front, lookat=lookat, up=up)

In [8]:
# Multi-order Ransac

max_plane_idx = 7
pt_to_plane_dist = 0.4

segment_models = {}
segments = {}
main_surface_idx = 0
largest_surface_points = 0
rest = pcd

for i in range(max_plane_idx):
    print(f'Run {i}/{max_plane_idx} started. ', end='')
    colors = plt.get_cmap("tab20")(i)
    segment_models[i], inliers = rest.segment_plane(distance_threshold=pt_to_plane_dist,ransac_n=3,num_iterations=50000)
    segments[i] = rest.select_by_index(inliers)
    if len(segments[i].points) > largest_surface_points:
        largest_surface_points = len(segments[i].points) 
        main_surface_idx = i
    segments[i].paint_uniform_color(list(colors[:3]))
    rest = rest.select_by_index(inliers, invert=True)
    print('Done')

print('Largest surface found with segment idx', main_surface_idx)
print(segment_models)
#o3d.visualization.draw_geometries([segments[i] for i in range(max_plane_idx)]+[rest],zoom=zoom,front=front,lookat=lookat,up=up)

Run 0/7 started. Done
Run 1/7 started. Done
Run 2/7 started. Done
Run 3/7 started. Done
Run 4/7 started. Done
Run 5/7 started. Done
Run 6/7 started. Done
Largest surface found with segment idx 0
{0: array([ 0.00143699,  0.00384227,  0.99999159, -0.74456263]), 1: array([ 0.00814197, -0.53006545,  0.84791764, -1.84750476]), 2: array([0.45026251, 0.27378575, 0.8498853 , 1.28167885]), 3: array([-0.01017461,  0.51948997,  0.85441597,  0.42343939]), 4: array([ 0.47051246, -0.24888749,  0.84656544, -0.29458513]), 5: array([-0.45046853, -0.26538736,  0.8524363 , -2.79663185]), 6: array([-0.46947973,  0.25628603,  0.84492973, -2.11099953])}


In [9]:
# #Initial attempt at gap detection - Idea: Chris , Execution: ChatGPT
# from tqdm import tqdm 

# main_surface_pcd = segments[main_surface_idx]

# # Create a KDTree for efficient neighbor searches
# pcd_tree = o3d.geometry.KDTreeFlann(main_surface_pcd)

# # Define grid bounds based on the point cloud extents
# min_bound = main_surface_pcd.get_min_bound()
# max_bound = main_surface_pcd.get_max_bound()

# # Set grid resolution and neighbor search parameters
# grid_resolution = 0.2  # Set based on your application's needs
# search_radius = 1    # Set based on average spacing/density of points
# neighbor_threshold = 5 # Threshold for minimum number of neighbors

# # Create a grid of points to sample
# x_grid = np.arange(min_bound[0], max_bound[0], grid_resolution)
# y_grid = np.arange(min_bound[1], max_bound[1], grid_resolution)
# z_value = np.mean([min_bound[2], max_bound[2]])  # Assuming a relatively flat surface on the XY plane

# gap_points = []
# for x in tqdm(x_grid):
#     for y in y_grid:
#         query_point = np.array([x, y, z_value])
#         k, idx, _ = pcd_tree.search_radius_vector_3d(query_point, search_radius)
#         if k < neighbor_threshold:
#             gap_points.append(query_point)

# # Create a point cloud from gap points
# gap_pcd = o3d.geometry.PointCloud()
# gap_pcd.points = o3d.utility.Vector3dVector(gap_points)
# gap_pcd.paint_uniform_color([1, 0, 0])  # Red color for gaps

# # Visualize the results
# o3d.visualization.draw_geometries([main_surface_pcd, gap_pcd], zoom=zoom, front=front, lookat=lookat, up=up)

In [15]:
main_surface_pcd = segments[main_surface_idx]
np.asarray(main_surface_pcd.points)[:, 2] = 0

pcd_tree = o3d.geometry.KDTreeFlann(main_surface_pcd)

grid_resolution = 0.2 
search_radius = 1  
neighbor_threshold = 10


min_bound = main_surface_pcd.get_min_bound()
max_bound = main_surface_pcd.get_max_bound()

x_grid = np.arange(min_bound[0], max_bound[0], grid_resolution)
y_grid = np.arange(min_bound[1], max_bound[1], grid_resolution)
z_value = np.mean([min_bound[2], max_bound[2]])

gap_points = []
for x in tqdm(x_grid):
    for y in y_grid:
        query_point = np.array([x, y, z_value])
        k, idx, _ = pcd_tree.search_radius_vector_3d(query_point, search_radius)
        if k < neighbor_threshold:
            gap_points.append(query_point)

gap_pcd = o3d.geometry.PointCloud()
gap_pcd.points = o3d.utility.Vector3dVector(gap_points)
gap_pcd.paint_uniform_color([1, 0, 0]) 

o3d.visualization.draw_geometries([main_surface_pcd, gap_pcd], zoom=zoom, front=front, lookat=lookat, up=up)

100%|██████████| 1248/1248 [00:09<00:00, 131.57it/s]
